[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Fgeolocate.ipynb
[Run in Google Colab]: https://colab.research.google.com/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/geolocate.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] | [![Colab logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/colab_logo.png)][Run in Google Colab] |
|:---------------------:|:---------------:|:-------------:|:-------------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] | [Run in Google Colab] |

## Introduction
This notebook provides an example of using the public REST API of WebAccess/DMP.

It plots the last-known location of one device, using cell tower information.

It uses [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) to display the interactive map.

It uses Google's [Geolocation API](https://developers.google.com/maps/documentation/geolocation/intro) to determine location from cell tower information.

### Requirements
* If you are running in Jupyter Notebook (>= version 5.3) you don't need to change any code.
* If you are running in Jupyter Lab, you will also need to install these JupyterLab extensions:
  * @jupyter-widgets/jupyterlab-manager
  * jupyter-leaflet
* You need to have an existing user account on the WA/DMP instance.
* In order to see something on the map, you need a device with a cellular connection.
* In order to plot location based on cell tower information, you must have an API key for Google's Geolocation API. (The Geolocation API is free if you make less than 40,000 calls per month).

### Usage
In the "Global Variables" cell below, change BASE_URL to match the particular WA/DMP instance that you are using.

Then run the cells, either one at a time, or all at once.

When prompted, enter the required User Input (USERNAME, PASSWORD, device_name, GOOGLE_KEY).

Cell tower location is also marked with a bounding circle. i.e. The device could be anywhere inside this circle.

## Setup
This may take a minute ...

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install ipyleaflet

import requests
import json
from ipyleaflet import Map, Marker, Circle

## Global variables

In [ ]:
BASE_URL = 'https://gateway.staging.wadmp.com'
BASE_PATH = 'api'
SESSION = requests.Session()  # Use one HTTPS session for all API calls

## User input

In [ ]:
USERNAME = input("Enter username:")
PASSWORD = input("Enter password:")
device_name = input("Enter device alias or MAC address:")

## Login

In [ ]:
url = f"{BASE_URL}/public/auth/connect/token"
credentials = {'username': USERNAME, 'password': PASSWORD, 'client_id': 'python', 'grant_type': 'password'}
print(f"\nSending POST request to {url} with:\n"
        f"    credentials={credentials}")
response = SESSION.post(url, data=credentials)

print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4, sort_keys=True))
except ValueError:
    print(response.text)

if response.status_code == requests.codes['ok']:
    user_token = response.json()["access_token"]
else:
    print("Failed to login!")
    sys.exit(1)

## Use the token in all subsequent API calls

In [ ]:
SESSION.headers.update({'Authorization': f'Bearer {user_token}'})

## Get the device details

In [ ]:
url = f"{BASE_URL}/{BASE_PATH}/management/devices"
query = {'page': 1, 'pageSize': '10', 'name': device_name}
print(f"\nSending GET request to {url} with:\n"
        f"    page=1\n"
        f"    pageSize=10\n"
        f"    name={device_name}\n")
response = SESSION.get(url, params=query)

print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4, sort_keys=True))
except ValueError:
    print(response.text)

if response.status_code == requests.codes['ok']:
    mac = response.json()['data'][0]['mac_address']
    company = response.json()['data'][0]['company']['name']
else:
    print("GET devices query failed!")
    sys.exit(1)

## Get the latest location data from InfluxDB
Note that the company name must be included in the WHERE clause!
(If you do not specify a company, the query will default to use the InfluxDB database for your primary company).

And because of the way Grafana [uses variables in queries](https://grafana.com/docs/grafana/latest/features/datasources/influxdb/#using-variables-in-queries), you have to wrap the company name as follows:
`"companyName" =~ /^My Company Inc.$/`

Following the Influx recommendations [here](https://docs.influxdata.com/influxdb/v1.7/troubleshooting/frequently-asked-questions/#when-should-i-single-quote-and-when-should-i-double-quote-in-queries), we single-quote string values and double-quote identifiers.

In [ ]:
url = f"{BASE_URL}/{BASE_PATH}/monitoring/devices/query"
influx_query = f'SELECT LAST("mobileTechnology"), LAST("mobilePLMN"), LAST("mobileLAC"), LAST("mobileCell") FROM "SNMP" WHERE ("macAddress" = \'{mac}\' AND "companyName" =~ /^{company}$/)'
query = {'Q': influx_query, 'Epoch': 'ms'}
print(f"\nSending GET request to {url} with:\n"
        f"    Q={influx_query}\n"
        f"    Epoch=ms\n")
response = SESSION.get(url, params=query)

print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4, sort_keys=True))
except ValueError:
    print(response.text)

if response.status_code == requests.codes['ok']:
    [timestamp, mobileTechnology, PLMN, locationAreaCode, cellId] = response.json()['results'][0]['series'][0]['values'][0]
    mobileCountryCode = PLMN[:3]
    mobileNetworkCode = PLMN[3:]
    print(f"mobileTechnology = {mobileTechnology}, " \
          f"mobileCountryCode = {mobileCountryCode}, " \
          f"mobileNetworkCode = {mobileNetworkCode}, " \
          f"locationAreaCode = {locationAreaCode}, " \
          f"cellId = {cellId}")
else:
    print("InfluxDB query failed!")
    sys.exit(1)

## Use Google's Geolocation API
See https://developers.google.com/maps/documentation/geolocation/intro

Note that when using `requests.post` with a `json` argument, the `Content-Type` header will be set to `application/json` automatically.

In [ ]:
GOOGLE_KEY = input("Enter your Google API key:")

url = "https://www.googleapis.com/geolocation/v1/geolocate"
query = {'key': GOOGLE_KEY}
body = {
  "cellTowers": [
    {
        "cellId": int(cellId, 16),
        "locationAreaCode": int(locationAreaCode, 16),
        "mobileCountryCode": mobileCountryCode,
        "mobileNetworkCode": mobileNetworkCode,
    }
  ]
}

print(f"\nSending POST request to {url} with:\n"
        f"    body={body}")
response = requests.post(url, params=query, json=body)

print(response.status_code)
try:
    print(json.dumps(response.json(), indent=4, sort_keys=True))
except ValueError:
    print(response.text)

if response.status_code == requests.codes['ok']:
    lat = response.json()['location']['lat']
    lng = response.json()['location']['lng']
    accuracy = response.json()['accuracy']
else:
    print("Google API query failed!")
    sys.exit(1)

The accuracy, in meters, represents the radius of a circle around the given location.

In [ ]:
centre = (lat, lng)

m = Map(center=centre, zoom=10)

circle = Circle()
circle.location = (lat, lng)
circle.radius = accuracy
m.add_layer(circle)

display(m)